In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import re
import dateutil.parser as dparser
from tqdm.notebook import tqdm
import tiktoken
import openai
import os
import time
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
OPENAI_KEY = os.getenv('OPENAI_KEY')
openai.api_key = OPENAI_KEY

In [3]:
def find_max_len(df):
    max_len = 0
    for text in df['text']:
        if len(text) > max_len:
            max_len = len(text)
    return max_len

## Dataframes

In [4]:
df_nine_for_news = pd.read_csv('nine_for_news_clean.csv').drop(['index'], axis=1).dropna()
df_nine_for_news['source'] = 'Nine for News'

# Filter all non-article entries
df_nine_for_news = df_nine_for_news[df_nine_for_news['url'].str.contains("2008|2009|2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020|2021|2022|2023|tag|categorie|page")==False]


df_nine_for_news

,text,date,url,source
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/felle-kritiek-op-fe...,Nine for News
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/bijwerkingencentrum...,Nine for News
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/internet-ontploft-n...,Nine for News
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/ggd-baas-heeft-spij...,Nine for News
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-europarlementa...,Nine for News
...,...,...,...,...
1356,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,18-10-2019,https://www.ninefornews.nl/stad-vandaan-kom-br...,Nine for News
1357,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,8-10-2019,https://www.ninefornews.nl/tientallen-ufos-wit...,Nine for News
1358,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,7-10-2019,https://www.ninefornews.nl/groot-boeing-737-ge...,Nine for News
1359,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,5-10-2019,https://www.ninefornews.nl/vloot-vreemde-objec...,Nine for News


In [5]:
df_indymedia= pd.read_csv('indymedia_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
df_indymedia['source'] = 'Indy media'

# Filter all non-article entries
df_indymedia = df_indymedia[df_indymedia['url'].str.contains("node")==True]
df_indymedia

,text,date,url,source
0,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,18-5-2023,https://www.indymedia.nl/node/53408,Indy media
1,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,16-5-2023,https://www.indymedia.nl/node/53390,Indy media
2,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-11-2023,https://www.indymedia.nl/node/53372,Indy media
3,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-6-2023,https://www.indymedia.nl/node/53366,Indy media
4,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-3-2023,https://www.indymedia.nl/node/53348,Indy media
...,...,...,...,...
57,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,5-10-2023,https://www.indymedia.nl/node/53371,Indy media
58,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,20-5-2023,https://www.indymedia.nl/node/53419,Indy media
59,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,16-5-2023,https://www.indymedia.nl/node/add/comment/5339...,Indy media
60,IndyMedia\n\n\n\n\n\n\n\n\n\nSteun Indymedia N...,16-5-2023,https://www.indymedia.nl/node/53395,Indy media


In [6]:
df_niburu = pd.read_csv('niburu_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
df_niburu['source'] = 'Niburu'

# Filter all non-article entries
df_niburu = df_niburu[df_niburu['url'].str.contains("start")==False]
df_niburu

,text,date,url,source
0,Toggle navigationInfo Nog steeds is het groots...,28-5-2023,https://niburu.co/,Niburu
1,Toggle navigationInfo Nog steeds is het groots...,28-5-2023,https://niburu.co/het-complot/18598-het-absurd...,Niburu
2,Toggle navigationInfo Je hebt in ons land twee...,28-5-2023,https://niburu.co/het-complot/18597-politie-bo...,Niburu
3,Toggle navigationInfo We vergeten snel en dat ...,28-5-2023,https://niburu.co/het-complot/18592-dit-is-hoe...,Niburu
4,Toggle navigationInfoHet heeft er alle schijn ...,27-5-2023,https://niburu.co/binnenland/18595-er-lijkt-ie...,Niburu
...,...,...,...,...
239,Toggle navigationInfo Op het moment dat kinder...,9-4-2023,https://niburu.co/het-complot/18448-scholen-wo...,Niburu
240,Toggle navigationInfo In de alternatieve werel...,8-4-2023,https://niburu.co/buitenland/18444-wordt-rober...,Niburu
241,Toggle navigationInfo Over het algemeen heeft ...,8-4-2023,https://niburu.co/buitenland/18446-het-ziet-er...,Niburu
242,Toggle navigationInfo Al jarenlang waarschuwen...,8-4-2023,https://niburu.co/het-complot/18445-nog-veel-m...,Niburu


### Privacynews doesn't really have any extreme content as it is only concerned with news about privacy

In [7]:
# df_privacynieuws = pd.read_csv('privacynieuws_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
# df_privacynieuws['source'] = 'Privacy nieuws'
# find_max_len(df_privacynieuws)

In [8]:
df_frontnieuws = pd.read_csv('frontnieuws_clean.csv').drop(['Unnamed: 0', 'index'], axis=1).dropna()
df_frontnieuws['source'] = 'Frontnieuws'

# Filter all non-article entries
df_frontnieuws= df_frontnieuws[df_frontnieuws['url'].str.contains("category|#comments|#respond|email-protection")==False]
df_frontnieuws

,text,date,url,source
1,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\nCOVID-1...,28-5-2023,https://www.frontnieuws.com/covid-19-cartoons/,Frontnieuws
7,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n - - A...,28-5-2023,https://www.frontnieuws.com/,Frontnieuws
8,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n529KMaa...,28-5-2023,https://www.frontnieuws.com/de-anti-antisemiti...,Frontnieuws
9,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\nFrontni...,28-5-2023,https://www.frontnieuws.com/author/frontnieuws...,Frontnieuws
10,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n768WAls...,28-5-2023,https://www.frontnieuws.com/is-oekraine-babe-r...,Frontnieuws
...,...,...,...,...
628,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1745Jam...,5-11-2020,https://www.frontnieuws.com/verkiezingsfraude-...,Frontnieuws
630,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1847()H...,26-1-2021,https://www.frontnieuws.com/pfizer-covid-19-va...,Frontnieuws
632,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n938CC B...,13-7-2020,https://www.frontnieuws.com/servische-soldaten...,Frontnieuws
634,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\nDe beëd...,7-10-2021,https://www.frontnieuws.com/luchtmacht-arts-vs...,Frontnieuws


# Skipping these for now as there is no date info available
The dataset is also quite large already. We don't want the GPT costs to be too high.

In [9]:
# df_de_andere_krant = pd.read_csv('results_de_andere_krant.csv').dropna()
# df_de_andere_krant
# find_max_len(df_de_andere_krant)

In [10]:
# df_transitieweb = pd.read_csv('results_transitieweb.csv').dropna()
# df_transitieweb
# find_max_len(df_transitieweb)

# Full dataset

In [11]:
frames = [df_nine_for_news, df_indymedia, df_niburu, df_frontnieuws]
final_df = pd.concat(frames, ignore_index=True)
final_df

,text,date,url,source
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/felle-kritiek-op-fe...,Nine for News
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/bijwerkingencentrum...,Nine for News
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/internet-ontploft-n...,Nine for News
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/ggd-baas-heeft-spij...,Nine for News
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-europarlementa...,Nine for News
...,...,...,...,...
1532,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1745Jam...,5-11-2020,https://www.frontnieuws.com/verkiezingsfraude-...,Frontnieuws
1533,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1847()H...,26-1-2021,https://www.frontnieuws.com/pfizer-covid-19-va...,Frontnieuws
1534,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n938CC B...,13-7-2020,https://www.frontnieuws.com/servische-soldaten...,Frontnieuws
1535,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\nDe beëd...,7-10-2021,https://www.frontnieuws.com/luchtmacht-arts-vs...,Frontnieuws


# GPT
## Tokenizing

In [12]:
def tokenize_text(df):
    for i, row in enumerate(df['text']):
        text = str(row)
        tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')
        encoded_text = tokenizer.encode(text)

        # Max amount of tokens for a request is a little more than 4000
        # this includes both the text and the instructions given to GPT
        # Lets remove the texts > 3000 tokens
        if len(encoded_text) > 3000:
            encoded_text = pd.NA

        df['encoded text'][i] = encoded_text

    return df

In [13]:
final_df['encoded text'] = ''
final_df = tokenize_text(final_df)

# Checking how many rows have a token count>3000
too_much_tokens = final_df['encoded text'].isna().sum()
print(f'Number of row > 3000 tokens: {too_much_tokens}')

# Dropping them
final_df = final_df.dropna()
final_df

Number of row > 3000 tokens: 163


,text,date,url,source,encoded text
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/felle-kritiek-op-fe...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/bijwerkingencentrum...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/internet-ontploft-n...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/ggd-baas-heeft-spij...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-europarlementa...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817..."
...,...,...,...,...,...
1528,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n2036HHe...,5-2-2023,https://www.frontnieuws.com/brits-leger-zou-in...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8..."
1531,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n2562LVa...,27-5-2023,https://www.frontnieuws.com/oekrainse-claim-ov...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8..."
1532,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1745Jam...,5-11-2020,https://www.frontnieuws.com/verkiezingsfraude-...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8..."
1533,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1847()H...,26-1-2021,https://www.frontnieuws.com/pfizer-covid-19-va...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8..."


In [14]:
def count_tokens_and_cost(df, role_prompt):
    # Counting the amount of tokens for the role prompt
    tokenizer = tiktoken.encoding_for_model('gpt-3.5-turbo')
    encoded_text = tokenizer.encode(role_prompt)

    count = 0
    for i in df['encoded text']:
        # Adding the role prompt as well since it is given with each row in the dataset
        count += len(i) + len(encoded_text)
    cost = (count/1000)*0.002
    return print(f'Amount of tokens = {count}, Price = {cost}$')

In [15]:
role_prompt = 'Je bent een robot die teksten analyseerd. Je antwoorden moeten in het Nederlands zijn. Haal de belangrijkste genoemde entiteiten en besproken onderwerpen uit de tekst. Kies maximaal 10 entiteiten en onderwerpen. Zet bij elk onderwerp en elke entiteit erbij of er positief of negatief over gepraat wordt in de tekst. Het is belangrijk dat je het sentiment van de auteur correct weergeeft. Geef antwoord als in het voorbeeld, dit is een Python dictionary: \n {entiteiten:{entiteit:sentiment}, onderwerpen:{onderwerp:sentiment}} De tekst: '

count_tokens_and_cost(final_df, role_prompt)

Amount of tokens = 2240027, Price = 4.480054$


In [16]:
def get_entities_and_subjects_gpt(text):
    messages = []
    messages.append({"role":"system", "content":'Je bent een robot die teksten analyseerd. Je antwoorden moeten in het Nederlands zijn.' })
    messages.append({"role": "user", "content": 'Haal de belangrijkste genoemde entiteiten en besproken onderwerpen uit de tekst. Kies maximaal 10 entiteiten en onderwerpen. Zet bij elk onderwerp en elke entiteit erbij of er positief of negatief over gepraat wordt in de tekst. Het is belangrijk dat je het sentiment van de auteur correct weergeeft. Geef antwoord als in het voorbeeld, dit is een Python dictionary: {entiteiten:{entiteit:sentiment}, onderwerpen:{onderwerp:sentiment}}'})
    messages.append({"role":"user", "content":f'De tekst: {text}'})
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages
    )
    messages = []
    gpt_result = response.choices[0].message['content']

    return gpt_result

In [17]:
final_df['GPT output'] = ''

In [18]:
final_df

,text,date,url,source,encoded text,GPT output
0,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/felle-kritiek-op-fe...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817...",
1,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/bijwerkingencentrum...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817...",
2,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/internet-ontploft-n...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817...",
3,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,28-05-2023,https://www.ninefornews.nl/ggd-baas-heeft-spij...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817...",
4,\n\nNet Binnen\nSwitch skinSidebarWillekeurig ...,27-05-2023,https://www.ninefornews.nl/kijk-europarlementa...,Nine for News,"[271, 7099, 426, 35811, 198, 17259, 6930, 4817...",
...,...,...,...,...,...,...
1528,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n2036HHe...,5-2-2023,https://www.frontnieuws.com/brits-leger-zou-in...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8...",
1531,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n2562LVa...,27-5-2023,https://www.frontnieuws.com/oekrainse-claim-ov...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8...",
1532,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1745Jam...,5-11-2020,https://www.frontnieuws.com/verkiezingsfraude-...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8...",
1533,\n\n\n\nWelcome!ZoekFrontnieuws\n\n\n\n1847()H...,26-1-2021,https://www.frontnieuws.com/pfizer-covid-19-va...,Frontnieuws,"[1038, 14262, 0, 57, 78, 1247, 24284, 11044, 8...",


In [19]:
def run_gpt_analysis(df):
    for index, text in enumerate(tqdm(df['text'])):
        for attempt in range(4):
            try:
                result = get_entities_and_subjects_gpt(text)
                df['GPT output'][index] = result
                print(f'Row {index} is done!')
                break
            except:
                print(f'Sleeping 16 sec at attempt {attempt} for row {index}.')
                time.sleep(16)
        df.to_csv('Final_results.csv')

In [ ]:
run_gpt_analysis(final_df)

  0%|          | 0/1374 [00:00<?, ?it/s]

Row 0 is done!
Row 1 is done!
Row 2 is done!
Row 3 is done!
Sleeping 16 sec at attempt 0 for row 4.
Row 4 is done!
Row 5 is done!
Sleeping 16 sec at attempt 0 for row 6.
Row 6 is done!
Sleeping 16 sec at attempt 0 for row 7.
Row 7 is done!
Sleeping 16 sec at attempt 0 for row 8.


In [ ]:
df = pd.read_csv('Final_results.csv')
df['GPT output']